Patching Array
===
***
Problem
---

https://leetcode.com/problems/patching-array/

Given a sorted positive integer array nums and an integer n, add/patch elements to the array such that any number in range [1, n] inclusive can be formed by the sum of some elements in the array. Return the minimum number of patches required.

**Example 1:**  

Input: nums = [1,3], n = 6  
Output: 1   
Explanation:  
Combinations of nums are [1], [3], [1,3], which form possible sums of: 1, 3, 4.  
Now if we add/patch 2 to nums, the combinations are: [1], [2], [3], [1,3], [2,3], [1,2,3].  
Possible sums are 1, 2, 3, 4, 5, 6, which now covers the range [1, 6].  
So we only need 1 patch.  


**Example 2:**  

Input: nums = [1,5,10], n = 20  
Output: 2  
Explanation: The two patches can be [2, 4].  


**Example 3:**  

Input: nums = [1,2,2], n = 5  
Output: 0  

In [83]:
import itertools
import random

**Brute force solution - \$O(2^n)\$**

In [87]:
def subsets(arr):
    if not arr:
        return [tuple()]
    first = arr.pop()
    rest = subsets(arr)
    incl = [(first,) + r for r in rest]
    return rest + incl

def is_ok(arr, n):
    sums = set(range(1, n + 1))
    for s in subsets(arr):
        sums.discard(sum(s))
    return not sums


def min_patches_bf(arr, n):
    arr = set(arr)
    missing = set(range(1, n + 1)) - arr
    best = missing
    for s in subsets(list(missing)):
        if is_ok(list(s) + list(arr), n) and len(s) < len(best):
            best = set(s)
    return list(best)

In [134]:
def min_patches(arr, n):
    N = n + 1
    
    initial = set()
    max_here = 0
    for x in sorted(arr):
        update = set()
        for i in initial:
            if i + x < N:
                update.add(i + x)
                max_here = max(max_here, i + x)
        if 0 < x < N:
            update.add(x)
            max_here = max(max_here, x)
        initial.update(update)

    min_missing = N
    for i in range(1, N):
        if i not in initial:
            min_missing = i
            break
    
    patches = 0
    while min_missing < max_here and min_missing < N:
        patches += 1
        update = set()
        for i in initial:
            if i + min_missing < N:
                update.add(i + min_missing)
                max_here = max(max_here, i + min_missing)
        update.add(min_missing)
        initial.update(update)
        min_missing = N
        for i in range(1, N):
            if i not in initial:
                min_missing = i
                break
    
    while min_missing < N:
        patches += 1
        max_here = 2 * min_missing - 1
        min_missing = 2 * min_missing

    return patches

In [132]:
def min_patches(arr, n):
    N = n + 1
    mask = [0] * N
    for x in sorted(arr):
        update = mask[:]
        for i in range(1, N):
            if mask[i] and i + x < N:
                update[i + x] = 1
        if 0 < x < N:
            update[x] = 1
        mask[:] = update

    patches = []
    while sum(mask) < n:
        # Find the minimum missing num
        for i in range(1, N):
            if not mask[i]:
                break
        patches.append(i)
        update = mask[:]
        for j in range(1, N):
            if mask[j] and i + j < N:
                update[j + i] = 1
        update[i] = 1
        mask[:] = update
    return patches

**Greedy solution - \$O(n)\$**

In [222]:
def smallest_missing(nums):
    """Returns the smallest positive number that cannot be
    represented as a sum of any subset in `nums`"""
    nums.sort()
    res = 1
    for i in nums:
        if i <= res:
            res += i
        else:
            return res
    return res


def is_ok(nums, n):
    return smallest_missing(nums) > n


def min_patches(nums, n):
    nums.sort()
    smallest_missing = 1
    patches = []
    i, N = 0, len(nums)
    while i < N:
        x = nums[i]
        if smallest_missing > n:
            break
        if x <= smallest_missing:
            smallest_missing += x
            i += 1
        else:
            patches.append(smallest_missing)
            smallest_missing *= 2

    while smallest_missing <= n:
        patches.append(smallest_missing)
        smallest_missing *=  2

    return patches


def min_patches_fast(nums, n):
    nums.sort()
    smallest_missing = 1
    patches = []
    i, N = 0, len(nums)
    while smallest_missing <= n:
        if i < N and nums[i] <= smallest_missing:
            smallest_missing += nums[i]
            i += 1
        else:
            patches.append(smallest_missing)
            smallest_missing *= 2

    return patches

**Tests**

In [220]:
assert(min_patches([1, 3], 6) == [2])
assert(min_patches([], 5) == [1, 2, 4])
assert(min_patches([], 1) == [1])
assert(min_patches([1], 2) == [2])
assert(min_patches([1, 1, 1], 5) == [4])
assert(min_patches([2, 2], 8) == [1, 6])
assert(min_patches([1, 5, 10], 20) == [2, 4])
assert(min_patches([1, 2, 2, 10], 35) == [6, 22])
assert(min_patches([0, 3], 7) == [1, 2, 7])


arr = list(range(10))
for i in range(1, 4):
    for test in itertools.combinations(arr, i):
        for n in range(1, 100):
            res = min_patches_fast(test, n)
            assert(is_ok(list(test) + res, n))


In [218]:
%timeit min_patches([random.randint(1, 100) for i in range(10)], 10000)

18.8 µs ± 29.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [224]:
%timeit min_patches_fast([random.randint(1, 100) for i in range(10)], 10000)

19 µs ± 117 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
